In [24]:
!nvidia-smi

Wed Apr 20 02:42:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    36W / 250W |   1149MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 6.6 MB 36.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from transformers import RobertaTokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

class CodeContestDataset(data.Dataset):
    """CodeContest dataset."""

    def __init__(self, csv_file, tokenizer):
        """
        Args:
            csv_file (string): Path to the csv file
        """
        self.data = pd.read_csv(csv_file)
        self.data = self.data[(self.data["Solution Language"] == "PYTHON3") & (self.data["Score"].notnull())]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.numpy()

        score = float(self.data.iloc[idx, 6])
        score = '{0:.2f}'.format(score)
        code = None
        if pd.isnull(self.data.iloc[idx, 5]):
          code = self.data.iloc[idx, 4]
        else:
          code = self.data.iloc[idx, 5]

        prefix_string = "Score: " + str(self.data.iloc[idx, 2])
        ids_object = self.tokenizer(prefix_string + "</s>" + str(code), return_tensors="pt", padding='max_length')
        input_ids = ids_object.input_ids[0]
        attention_mask = ids_object.attention_mask[0]

        output_ids_object = self.tokenizer(score, return_tensors="pt")
        output_ids = output_ids_object.input_ids[0]
        output_attention_mask = output_ids_object.attention_mask[0]

        sample = {"source_ids": input_ids, "source_mask": attention_mask, "target_ids": output_ids, "target_mask": output_attention_mask}
        return sample

In [28]:
file_names = []
for filename in os.listdir("./DeepLearningFinalProject/CodeContestsClean"):
  file_names.append(filename)
file_names.sort()

In [29]:
all_datasets = []
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
for filename in file_names:
  all_datasets.append(CodeContestDataset("./DeepLearningFinalProject/CodeContestsCleanCodeBLEU/" + filename, tokenizer))

all_data = data.ConcatDataset(all_datasets)
dataset = data.ConcatDataset(all_datasets[:150])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


In [30]:
len(all_data)

1045739

In [31]:
len(dataset)

783889

In [10]:
for i in range(5):
  sample = dataset[i]
  print(tokenizer.decode(sample["source_ids"]))

<s>Score: One cold winter evening Alice and her older brother Bob was sitting at home near the fireplace and giving each other interesting problems to solve. When it was Alice's turn, she told the number n to Bob and said:

—Shuffle the digits in this number in order to obtain the smallest possible number without leading zeroes.

—No problem! — said Bob and immediately gave her an answer.

Alice said a random number, so she doesn't know whether Bob's answer is correct. Help her to find this out, because impatient brother is waiting for the verdict.

Input

The first line contains one integer n (0 ≤ n ≤ 109) without leading zeroes. The second lines contains one integer m (0 ≤ m ≤ 109) — Bob's answer, possibly with leading zeroes.

Output

Print OK if Bob's answer is correct and WRONG_ANSWER otherwise.

Examples

Input

3310
1033


Output

OK


Input

4
5


Output

WRONG_ANSWER</s>def sort(s):
    return sorted(sorted(s), key=str.upper)
s=input()
s1=input()
l=sort(s)
c=l.count('0')
res="

In [3]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 62.9 MB/s 
     |████████████████████████████████| 408 kB 57.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 56.3 MB/s 
     |████████████████████████████████| 144 kB 61.0 MB/s 


In [4]:
import pytorch_lightning as pl
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams, train_dataset, model):
    super(T5FineTuner, self).__init__()
    self.hparams.update(vars(hparams))
    self.train_dataset = train_dataset
    self.model = model
    self.tokenizer = RobertaTokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return True
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    #tensorboard_logs = {"avg_train_loss": avg_train_loss}
    self.log("avg_train_loss", avg_train_loss)

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                    batch_idx=None, 
                    optimizer=None, 
                    optimizer_idx=None, 
                    optimizer_closure=None, 
                    on_tpu=None, 
                    using_native_amp=None, 
                    using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    dataloader = data.DataLoader(self.train_dataset, batch_size=self.hparams.train_batch_size, shuffle=True, num_workers=6, pin_memory=True)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = data.ConcatDataset(all_datasets[150:])
    return data.DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [5]:
import logging
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [6]:
args_dict = dict(
    data_dir="",
    output_dir="./DeepLearningFinalProject",
    model_name_or_path='Salesforce/codet5-small',
    tokenizer_name_or_path='Salesforce/codet5-small',
    max_seq_length=512,
    learning_rate=1e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=16,
    eval_batch_size=16,
    num_train_epochs=1,
    gradient_accumulation_steps=8,
    n_gpu=1,
    fp_16=False,
    max_grad_norm=1.0,
    seed=42,
)

In [7]:
import argparse
args = argparse.Namespace(**args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

In [22]:
#The below code is execute 3 times for 3 splits of the training data, due to system RAM constraints in Colab
model_base = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')
trainer = pl.Trainer(**train_params)
curr_dataset = data.ConcatDataset(all_datasets[100:150])
model = T5FineTuner(args, curr_dataset, model_base)
model.load_state_dict(torch.load("./DeepLearningFinalProject/model_codet5_small_codeBLEU_python3_04192022_split1.pt"))
trainer.fit(model)
torch.save(model.state_dict(), "./DeepLearningFinalProject/model_codet5_small_codeBLEU_python3_04192022_split2.pt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f9f25b27bd0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f9f25b27bd0>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every opti

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [53]:
#The below code evaluates the mean absolute error on the validation data split
from sklearn import metrics
model.model.eval()
model.model.to(device)
test_dataset = data.ConcatDataset(all_datasets[150:])
loader = data.DataLoader(test_dataset, batch_size=16, num_workers=4)
actual = []
predicted = []
for batch_idx, batch in enumerate(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].to(device), 
                              attention_mask=batch['source_mask'].to(device), 
                              max_length=6)
  dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
  targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['target_ids']]
  actual.extend(targets)
  predicted.extend(dec)

predicted = [float(x) for x in predicted]
actual = [float(x) for x in actual]
metrics.mean_absolute_error(actual, predicted)

0.16228726370059193